# CPU ANN vs Exact: Performance and Accuracy

Compares `NNdex` CPU exact search (`approx=False`) against ANN-prefiltered search (`approx=True`) with cache disabled for fair timing.

In [ ]:
from __future__ import annotations

import time
import numpy as np

from nndex import NNdex


In [ ]:
rng = np.random.default_rng(7)
rows = 50_000
dims = 256
query_rows = 512
k = 25

matrix = rng.normal(size=(rows, dims)).astype(np.float32)
queries = rng.normal(size=(query_rows, dims)).astype(np.float32)


In [ ]:
def benchmark_batch(model: NNdex, q: np.ndarray, topk: int, repeats: int = 5) -> tuple[np.ndarray, np.ndarray, float]:
    timings_ms: list[float] = []
    out_idx = out_scores = None
    for _ in range(repeats):
        t0 = time.perf_counter()
        out_idx, out_scores = model.search(q, k=topk)
        timings_ms.append((time.perf_counter() - t0) * 1_000.0)
    assert out_idx is not None and out_scores is not None
    return out_idx, out_scores, float(np.median(timings_ms))

def topk_overlap_fraction(exact_idx: np.ndarray, approx_idx: np.ndarray) -> float:
    overlaps = []
    for exact_row, approx_row in zip(exact_idx, approx_idx):
        overlaps.append(len(set(exact_row.tolist()) & set(approx_row.tolist())) / exact_row.size)
    return float(np.mean(overlaps))

def mae_scores(exact_scores: np.ndarray, approx_scores: np.ndarray) -> float:
    return float(np.mean(np.abs(exact_scores - approx_scores)))


In [ ]:
exact = NNdex(matrix, normalized=False, approx=False, backend="cpu", enable_cache=False)
ann = NNdex(matrix, normalized=False, approx=True, backend="cpu", enable_cache=False)

exact_idx, exact_scores, exact_ms = benchmark_batch(exact, queries, k)
ann_idx, ann_scores, ann_ms = benchmark_batch(ann, queries, k)

print(f"Exact median batch latency: {exact_ms:.2f} ms")
print(f"ANN median batch latency:   {ann_ms:.2f} ms")
print(f"Speedup (Exact/ANN):        {exact_ms / ann_ms:.2f}x")
print()
print(f"Mean top-k overlap:         {topk_overlap_fraction(exact_idx, ann_idx):.3f}")
print(f"Mean absolute score error:  {mae_scores(exact_scores, ann_scores):.6f}")
